In [50]:
import numpy as np
import itertools
import time

n = 5  # Размерность функции - не 10 т.к. слишком вычислительно сложно

x_values_f1 = np.linspace(0.0, 1.0, 11)  # Возможные значения переменных для f1
all_points_f1 = list(itertools.product(x_values_f1, repeat=n))  # Все сочетания для f1

x_values_f2 = np.linspace(1.0, 2.0, 11)  # Возможные значения переменных для f2
all_points_f2 = list(itertools.product(x_values_f2, repeat=n))  # Все сочетания для f2

# Практика

In [61]:
class DualNumber:
    def __init__(self, real, dual=0):
        self.real = real
        self.dual = dual

    def __add__(self, other):
        if isinstance(other, DualNumber):
            return DualNumber(self.real + other.real, self.dual + other.dual)
        else:
            return DualNumber(self.real + other, self.dual)

    def __radd__(self, other):
        return self + other

    def __mul__(self, other):
        if isinstance(other, DualNumber):
            return DualNumber(self.real * other.real,
                              self.real * other.dual + self.dual * other.real)
        else:
            return DualNumber(self.real * other, self.dual * other)

    def __rmul__(self, other):
        return self * other

    def __pow__(self, power):
        if isinstance(power, (int, float)):
            # Обычное число в степени
            return DualNumber(self.real**power, power * self.real**(power - 1) * self.dual)
        elif isinstance(power, DualNumber):
            if not isinstance(self.real, (int, float)):
                raise TypeError("Base (self.real) must be a number when using DualNumber as power.")
            if self.real <= 0:
                raise ValueError("Base (self.real) must be positive for logarithm in DualNumber.")

            # Реальная часть результата
            real_part = self.real**power.real

            # Производная части: log работает только для вещественных чисел
            log_base = np.log(self.real)
            dual_part = real_part * (power.dual * log_base + power.real * self.dual / self.real)

            return DualNumber(real_part, dual_part)
        else:
            raise TypeError("Unsupported type for power: {}".format(type(power)))


    def __repr__(self):
        return f"DualNumber(real={self.real}, dual={self.dual})"

def evaluate_gradient_dual_number(func, point):
    n = len(point)
    result = []
    for i in range(n):
        dual_point = [DualNumber(val, 1 if j == i else 0) for j, val in enumerate(point)]
        output = func(*dual_point)
        result.append(output.dual)
    return func(*[DualNumber(x, 0) for x in point]).real, result

In [59]:
def f1(*args):
    return sum(x**2 for x in args)

# Вычисления для f1
start_time = time.time()

results = []
for point in all_points_f1:
    value, gradients = evaluate_gradient_dual_number(f1, point)
    results.append((point, value, gradients))

end_time = time.time()

# Результаты
print(f"Время выполнения: {end_time - start_time:.2f} секунд")
print(f"Пример результата для точки {all_points[len(results)//3]}")
print(f"Значение функции: {results[len(results)//3][1]}")
print(f"Градиенты: {results[len(results)//3][2]}")

Время выполнения: 13.09 секунд
Пример результата для точки (1.3, 1.7000000000000002, 1.3, 1.7000000000000002, 1.3)
Значение функции: 1.2500000000000002
Градиенты: [0.6000000000000001, 1.4000000000000001, 0.6000000000000001, 1.4000000000000001, 0.6000000000000001]


In [64]:
def f2(*args):
    return sum(x**x for x in args)

# Вычисления для f2
start_time = time.time()

results = []
for point in all_points_f2:
    value, gradients = evaluate_gradient_dual_number(f2, point)
    results.append((point, value, gradients))

end_time = time.time()

# Результаты
print(f"Время выполнения: {end_time - start_time:.2f} секунд")
print(f"Пример результата для точки {all_points[len(results)//3]}")
print(f"Значение функции: {results[len(results)//3][1]}")
print(f"Градиенты: {results[len(results)//3][2]}")

Время выполнения: 20.11 секунд
Пример результата для точки (1.3, 1.7000000000000002, 1.3, 1.7000000000000002, 1.3)
Значение функции: 9.148759818683398
Градиенты: [1.7754606438173388, 3.77253164340038, 1.7754606438173388, 3.77253164340038, 1.7754606438173388]


# ДЗ

In [65]:
class Node:
    def __init__(self, value, parents=None, grad_fn=None):
        """
        Узел вычислительного графа.
        :param value: Значение функции в этом узле.
        :param parents: Список родительских узлов.
        :param grad_fn: Функция для вычисления локальных градиентов.
        """
        self.value = value
        self.parents = parents if parents else []
        self.grad_fn = grad_fn
        self.grad = 0  # Градиент, который будет вычислен в процессе backward

    def backward(self, grad=1.0):
        """
        Обратный проход для вычисления градиентов.
        :param grad: Градиент от текущего узла.
        """
        self.grad += grad
        if self.grad_fn and self.parents:
            local_grads = self.grad_fn(grad)
            for parent, local_grad in zip(self.parents, local_grads):
                parent.backward(local_grad)

    def __add__(self, other):
        if not isinstance(other, Node):
            other = Node(other)
        value = self.value + other.value
        return Node(value, [self, other], lambda g: [g, g])

    def __mul__(self, other):
        if not isinstance(other, Node):
            other = Node(other)
        value = self.value * other.value
        return Node(value, [self, other], lambda g: [g * other.value, g * self.value])

    def __pow__(self, power):
        if isinstance(power, (int, float)):
            # Если степень - число
            value = self.value**power
            return Node(value, [self], lambda g: [g * power * self.value**(power - 1)])
        elif isinstance(power, Node):
            # Если степень - объект Node
            value = self.value**power.value
            log_base = np.log(self.value) if self.value > 0 else 0  # Проверяем положительность
            return Node(value, [self, power], lambda g: [
                g * power.value * self.value**(power.value - 1),  # Производная по основанию
                g * value * log_base                             # Производная по степени
            ])
        else:
            raise TypeError("Unsupported type for power: {}".format(type(power)))

    def __repr__(self):
        return f"Node(value={self.value}, grad={self.grad})"


def evaluate_gradient_backward_mode(func, point):
    """
    Вычисляет значение функции и градиенты через backward mode autodiff.
    :param func: Функция для анализа.
    :param point: Точка, в которой нужно вычислить значение и градиенты.
    :return: Значение функции и градиенты.
    """
    # Создаем узлы для входных значений
    nodes = [Node(x) for x in point]

    # Вычисляем значение функции
    result = func(*nodes)

    # Обратный проход для вычисления градиентов
    result.backward()

    # Градиенты - это накопленные значения в каждом входном узле
    gradients = [node.grad for node in nodes]
    return result.value, gradients

In [66]:
def f1(*args):
    initial_node = Node(0)
    for x in args:
        initial_node = initial_node + (x ** 2)  # Use Node's __add__ and __pow__
    return initial_node

# Вычисления для f1
start_time = time.time()

results = []
for point in all_points_f1:
    value, gradients = evaluate_gradient_backward_mode(f1, point)
    results.append((point, value, gradients))

end_time = time.time()

# Результаты
print(f"Время выполнения: {end_time - start_time:.2f} секунд")
print(f"Пример результата для точки {all_points[len(results)//3]}")
print(f"Градиенты: {results[len(results)//3][2]}")

Время выполнения: 5.38 секунд
Пример результата для точки (1.3, 1.7000000000000002, 1.3, 1.7000000000000002, 1.3)
Градиенты: [0.6000000000000001, 1.4000000000000001, 0.6000000000000001, 1.4000000000000001, 0.6000000000000001]


In [67]:
def f2(*args):
    initial_node = Node(0)
    for x in args:
        initial_node = initial_node + (x ** x)  # Use Node's __add__ and __pow__
    return initial_node

# Вычисления для f1
start_time = time.time()

results = []
for point in all_points_f2:
    value, gradients = evaluate_gradient_backward_mode(f2, point)
    results.append((point, value, gradients))

end_time = time.time()

# Результаты
print(f"Время выполнения: {end_time - start_time:.2f} секунд")
print(f"Пример результата для точки {all_points[len(results)//3]}")
print(f"Градиенты: {results[len(results)//3][2]}")

Время выполнения: 6.18 секунд
Пример результата для точки (1.3, 1.7000000000000002, 1.3, 1.7000000000000002, 1.3)
Градиенты: [1.7754606438173388, 3.772531643400379, 1.7754606438173388, 3.772531643400379, 1.7754606438173388]
